In [2]:
from pathlib import Path
import pandas as pd

In [13]:
dirpath = Path("/home/andreas/Workspace/github/semsim/tmp/Psycho-Paper/semD")
filename = "tdm.mm"
filepath = dirpath / filename
mm = pd.read_csv(filepath, sep=' ', comment='%')
mm

,74100,112798,16323790
0,51,1,0.573678
1,115,1,0.601497
2,171,1,0.300871
3,186,1,0.804711
4,208,1,0.296410
...,...,...,...
16323785,59391,112798,0.388743
16323786,60722,112798,0.361460
16323787,65987,112798,0.160685
16323788,66170,112798,0.235783


In [25]:
c = mm.columns
mm = mm[[c[1], c[0], c[2]]]
mm

,112798,74100,16323790
0,1,51,0.573678
1,1,115,0.601497
2,1,171,0.300871
3,1,186,0.804711
4,1,208,0.296410
...,...,...,...
16323785,112798,59391,0.388743
16323786,112798,60722,0.361460
16323787,112798,65987,0.160685
16323788,112798,66170,0.235783


In [32]:
filename = "tdm_gensim_format.mm"
filepath = dirpath / filename
with open(filepath, 'w') as fp:
    fp.write("%%MatrixMarket matrix coordinate real general\n")
    mm.to_csv(fp, index=None, sep=' ')


In [18]:
dirpath = Path("/home/andreas/Workspace/github/semsim/tmp/Psycho-Paper/semD")
filename = "tdm_transpose.mm"
filepath = dirpath / filename
mm_t = pd.read_csv(filepath, sep=' ', comment='%')
mm_t

,112798,74100,16323790
0,133,1,1.657571
1,139,1,0.552524
2,156,1,1.105047
3,212,1,0.552524
4,276,1,0.875729
...,...,...,...
16323785,49644,74100,0.782991
16323786,57199,74100,0.782991
16323787,69170,74100,0.782991
16323788,70372,74100,0.782991


In [24]:
mm_t.sort_values([mm_t.columns[0], mm_t.columns[1]])

,112798,74100,16323790
4492,1,51,0.573678
9644,1,115,0.601497
14090,1,171,0.300871
19897,1,186,0.804711
21233,1,208,0.296410
...,...,...,...
12781116,112798,59391,0.388743
13042485,112798,60722,0.361460
14221492,112798,65987,0.160685
14379297,112798,66170,0.235783


In [29]:
from gensim.corpora import MmCorpus
from semsim.metric.semantic_diversity import lsi_projection_sklearn

In [33]:
file_path = "/home/andreas/Workspace/github/semsim/tmp/Psycho-Paper/semD/tdm_gensim_format.mm"
corpus = MmCorpus(str(file_path))
corpus

In [34]:
_, document_vectors, _ = lsi_projection_sklearn(corpus, 300)

Size of train_set=112798
Training LSI model with 300 topics


In [35]:
assert len(document_vectors) == len(corpus)

# --- save document vectors ---
file_path = dirpath / f'tdm_lsi_sklearn_document_vectors.csv'
print(f"Saving document vectors to {file_path}")
document_vectors.to_csv(file_path)


Saving document vectors to /home/andreas/Workspace/github/semsim/tmp/Psycho-Paper/semD/tdm_lsi_sklearn_document_vectors.csv


In [36]:
document_vectors

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,5.661226,1.597181,-0.288954,1.787526,-0.249992,1.454049,0.202691,-0.607079,1.605152,0.194739,...,0.366571,0.245880,0.107692,0.164634,0.229716,-0.451767,-0.220531,-0.143977,0.246120,0.109206
1,7.134194,5.034952,2.597043,-2.629958,0.850340,1.944199,1.464639,-0.227598,-1.601632,0.692687,...,-0.528127,-0.300785,-0.107675,-0.189313,0.422372,0.282033,0.243802,-0.256624,0.598067,0.265258
2,7.711022,4.189820,1.736556,0.328536,0.225220,2.491375,-0.409075,0.050282,0.161171,-0.230135,...,0.327700,-0.198290,-0.496288,0.206674,-0.155514,-0.285168,0.602123,0.015196,-0.466119,-0.072746
3,0.756073,0.472406,0.099208,-0.377461,-0.091714,0.134928,-0.164262,-0.367878,-0.065391,-0.064810,...,-0.051284,-0.027606,-0.034305,0.007671,0.056982,-0.037480,0.011550,0.029284,0.023917,-0.102807
4,10.301138,-1.185438,3.300109,2.691346,-1.272054,-0.259552,-0.815333,1.551024,-1.144578,1.737632,...,0.008639,-0.632389,-0.089778,0.242389,0.579892,0.031460,1.034839,1.017614,0.470827,-0.606302
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112793,1.256355,-0.361180,0.414875,0.495926,-0.193516,-0.172503,-0.739369,-0.353278,-0.410653,-0.485344,...,0.022294,0.028797,0.025558,0.036348,-0.019562,-0.003106,-0.030167,0.018030,0.058483,-0.091241
112794,0.204222,-0.033498,-0.107541,-0.074891,-0.099967,-0.102337,-0.101516,-0.084214,0.005515,-0.008868,...,-0.044010,0.071205,-0.031642,0.041339,-0.045304,0.106335,0.070376,0.032448,0.050230,-0.028708
112795,0.226754,-0.100170,-0.156836,0.035769,-0.082209,0.096346,-0.009232,-0.094390,0.037919,-0.223809,...,0.004002,0.035046,-0.026256,0.012595,0.001848,-0.007630,0.002280,-0.028725,-0.001889,-0.016676
112796,0.155547,0.007962,-0.047608,-0.120524,-0.078884,-0.061303,-0.094548,-0.118931,-0.016058,-0.036082,...,0.027126,0.019838,0.016557,0.039501,-0.030715,-0.021141,0.014950,-0.013626,-0.002612,0.035704
